In [226]:
import pandas as pd
import numpy as np
from scipy import stats
import sklearn
from sklearn import preprocessing

In [217]:
df = pd.read_csv('/Users/tianchiz/Desktop/Aviana project/Aviana/train_set.csv')

/Users/tianchiz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [227]:
#function to do z-score transformation of a column
def zscore(column):
    if column_type(column) == 'continuous':
        return(stats.zscore(column))
    else:
        return(column)

In [228]:
#function to do min-max transformation of a column
def minmax(column):
    if column_type(column) == 'continuous':
        return(preprocessing.minmax_scale(column, feature_range=(0, 100)))
    else:
        return(column)

In [229]:
#function to do Box-Cox transformation of continuous target
def boxcox(target):
    if column_type(target) == 'continuous':
        c = min(target) - 1
        target1 = target - c
        target1 = stats.boxcox(target1)[0]
        return(zscore(target1))
    else:
        return(target)

In [230]:
# some necessary functions
def get_distinct_num(column): #input the whole column
    return len(column.unique().tolist())

def get_valid_num(column): #input the whole column
    return len(column) - get_na_num(column)

def get_na_num(column): #input the whole column
    if column.dtype == np.object:
        return column.isnull().sum() + column[column == ''].count() + column[column == '?'].count()
    else:
        return column.isnull().sum()

def check_cont_or_dis(column):
    if get_distinct_num(column) > max(0.0001 * get_valid_num(column), 10): #if relative is bigger than the threshold, we consider it to be continuous 
        return ('continuous')
    else:
        return ('discrete')

def column_type(column):
    if column.dtype == np.float64: #float type
        return ('continuous')
    elif column.dtype == np.int64: #int type
        if check_cont_or_dis(column) == 'discrete':
            return ('ordinal')
        else: 
            return ('continuous')
    else: #string type and others
        class1 = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', "I", 'J', 'K', 'L', 'M', 'N', 'MISSING']
        class2 = ['SSSS', 'SSS', 'SS', 'S', 'A', 'B', 'C', 'D', "E", 'F', 'G', 'H', "I", 'J', 'MISSING']
        unique_value = [str(x).upper() for x in column.unique().tolist()]
        count_class1 = 0 
        count_class2 = 0
        for i in unique_value:
            if (i in class1):
                count_class1 += 1
            elif (i in class2):
                count_class1 += 1
            else:
                count_class1 = 0
                count_class2 = 0
        if ((count_class1 <= 15 and count_class1 != 0) or (count_class2 <= 15 and count_class2 != 0)):
            return ('cat_ordinal')
        else:
            return ('nominal')
  

In [231]:
# test of boxcox transformation

target = df.loc[1:1000, 'Claim_Amount']
print(target)
target1 = boxcox(target)
print(target1)
print('max before boxcox transforfation=', max(target))
print('max after boxcox transforfation=', max(target1))

1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
971     0.0
972     0.0
973     0.0
974     0.0
975     0.0
976     0.0
977     0.0
978     0.0
979     0.0
980     0.0
981     0.0
982     0.0
983     0.0
984     0.0
985     0.0
986     0.0
987     0.0
988     0.0
989     0.0
990     0.0
991     0.0
992     0.0
993     0.0
994     0.0
995     0.0
996     0.0
997     0.0
998     0.0
999     0.0
1000    0.0
Name: Claim_Amount, Length: 1000, dtype: float64
[-0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378
 -0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378
 -0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378 -0.10050378


In [232]:
# test of minmax transformation

test1 = df.loc[1:1000, ['Vehicle', 'Blind_Model', 'Cat3', 'OrdCat', 'Var5', 'NVVar2']]
test2 = test1
print(test1)
for c in test1:
    test1[c] = minmax(test1[c])
print(test1)

      Vehicle Blind_Model Cat3 OrdCat      Var5    NVVar2
1           2        Q.22    A      5  1.240851 -0.266117
2           1       AR.41    A      2 -0.971487 -0.266117
3           1       AR.41    A      2 -0.971487 -0.266117
4           2        D.20    B      5  0.812656 -0.266117
5           2        D.20    B      5  0.812656 -0.266117
6           1      AJ.129    A      4  0.580718 -0.266117
7           2       AQ.17    B      4  0.527193 -0.266117
8           3       AQ.17    B      4  0.527193 -0.266117
9           1        BW.3    E      4  0.176312 -0.266117
10          1        BW.3    E      4  0.176312 -0.266117
11          3      BW.167    F      5  0.342832 -0.266117
12          3      BW.167    F      5  0.342832 -0.266117
13          4         Y.9    A      2 -0.608711 -0.266117
14          4         Y.9    A      2 -0.608711 -0.266117
15          1       BH.29    A      4 -0.763337 -0.266117
16          1       BH.29    A      4 -0.763337 -0.266117
17          2 

In [233]:
# test of zscore transformation

for c in test2:
    test2[c] = zscore(test2[c])
print(test2)

      Vehicle Blind_Model Cat3 OrdCat      Var5    NVVar2
1           2        Q.22    A      5  0.917664 -0.299284
2           1       AR.41    A      2 -1.131477 -0.299284
3           1       AR.41    A      2 -1.131477 -0.299284
4           2        D.20    B      5  0.521056 -0.299284
5           2        D.20    B      5  0.521056 -0.299284
6           1      AJ.129    A      4  0.306227 -0.299284
7           2       AQ.17    B      4  0.256651 -0.299284
8           3       AQ.17    B      4  0.256651 -0.299284
9           1        BW.3    E      4 -0.068347 -0.299284
10          1        BW.3    E      4 -0.068347 -0.299284
11          3      BW.167    F      5  0.085889 -0.299284
12          3      BW.167    F      5  0.085889 -0.299284
13          4         Y.9    A      2 -0.795462 -0.299284
14          4         Y.9    A      2 -0.795462 -0.299284
15          1       BH.29    A      4 -0.938682 -0.299284
16          1       BH.29    A      4 -0.938682 -0.299284
17          2 